In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)

Mounted at /content/gdrive


In [ ]:
import tensorflow as tf

!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 32kB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 3.8MB 31.1MB/s 
     |████████████████████████████████| 512kB 43.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=64a856dce8137e0c7f2b08d8c8cf6cc0ff53e54deafe0303ae5e9c01dba32723
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installati

In [ ]:
import tensorflow
print(tensorflow.__version__)


1.15.0


In [ ]:
import csv
from numpy import moveaxis
from numpy import asarray
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import cv2
from PIL import Image
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
#from tensorflow.python.keras.utils import np_utils
import matplotlib.image as mpimg

In [ ]:
cd '/content/gdrive/My Drive/Colab Notebooks/cleverhans'

/content/gdrive/My Drive/Colab Notebooks/cleverhans


In [ ]:
pwd

'/content/gdrive/My Drive/Colab Notebooks/cleverhans'

In [ ]:
import cleverhans

In [ ]:
# Dont run again
   # !pip install tensorflow==1.13.2

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging
import numpy as np
from six.moves import xrange
import tensorflow as tf

from cleverhans.attacks import SaliencyMapMethod
from cleverhans.compat import flags
from cleverhans.dataset import MNIST
from cleverhans.loss import CrossEntropy
from cleverhans.utils import other_classes, set_log_level
from cleverhans.utils import pair_visual, grid_visual, AccuracyReport
from cleverhans.utils_tf import model_eval, model_argmax
from cleverhans.train import train
from cleverhans.model_zoo.basic_cnn import ModelBasicCNN


In [ ]:
"""
This tutorial shows how to generate adversarial examples
using JSMA in white-box setting.
The original paper can be found at:
https://arxiv.org/abs/1511.07528
"""
#pylint: disable=missing-docstring
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
import time
import logging
import numpy as np
from six.moves import xrange
import tensorflow as tf
import copy

from cleverhans.attacks import SaliencyMapMethod
from cleverhans.compat import flags
from cleverhans.dataset import MNIST
from cleverhans.loss import CrossEntropy
from cleverhans.utils import other_classes, set_log_level
from cleverhans.utils import pair_visual, grid_visual, AccuracyReport
from cleverhans.utils_tf import model_eval, model_argmax
from cleverhans.train import train
from cleverhans.model_zoo.basic_cnn import ModelBasicCNN





In [ ]:
FLAGS = flags.FLAGS

VIZ_ENABLED = True
NB_EPOCHS = 6
BATCH_SIZE = 128
LEARNING_RATE = .001
SOURCE_SAMPLES = 10

train_start=0
train_end=60000 
test_start=0
test_end=10000 
viz_enabled=VIZ_ENABLED
learning_rate=LEARNING_RATE
def mnist_tutorial_jsma(train_start=0, train_end=60000, test_start=0,
                        test_end=10000, viz_enabled=VIZ_ENABLED,
                        nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE,
                        source_samples=SOURCE_SAMPLES,
                        learning_rate=LEARNING_RATE):
  """
  MNIST tutorial for the Jacobian-based saliency map approach (JSMA)
  :param train_start: index of first training set example
  :param train_end: index of last training set example
  :param test_start: index of first test set example
  :param test_end: index of last test set example
  :param viz_enabled: (boolean) activate plots of adversarial examples
  :param nb_epochs: number of epochs to train model
  :param batch_size: size of training batches
  :param nb_classes: number of output classes
  :param source_samples: number of test inputs to attack
  :param learning_rate: learning rate for training
  :return: an AccuracyReport object
  """
  # Object used to keep track of (and return) key accuracies
report = AccuracyReport()

  # Set TF random seed to improve reproducibility
tf.set_random_seed(1234)

  # Create TF session and set as Keras backend session
sess = tf.Session()
print("Created TensorFlow session.")

set_log_level(logging.DEBUG)

  # Get MNIST test data
mnist = MNIST(train_start=train_start, train_end=train_end,test_start=test_start, test_end=test_end)
x_train, y_train = mnist.get_set('train')
x_test, y_test = mnist.get_set('test')

  # Obtain Image Parameters
img_rows, img_cols, nchannels = x_train.shape[1:4]
nb_classes = y_train.shape[1]

  # Define input TF placeholder
x = tf.placeholder(tf.float32, shape=(None, img_rows, img_cols,nchannels))
y = tf.placeholder(tf.float32, shape=(None, nb_classes))

nb_filters = 64
  # Define TF model graph
model = ModelBasicCNN('model1', nb_classes, nb_filters)
preds = model.get_logits(x)
loss = CrossEntropy(model, smoothing=0.1)
print("Defined TensorFlow model graph.")


Created TensorFlow session.


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.

Defined TensorFlow model graph.


In [ ]:
  #########################################################################
  # Training the model using TensorFlow
  ###########################################################################

  # Train an MNIST model
  reuse=tf.AUTO_REUSE
  reuse=True
  train_params = {
      'nb_epochs': 10,
      'batch_size': 128,
      'learning_rate': 0.001
  }
  sess.run(tf.global_variables_initializer())
  rng = np.random.RandomState([2017, 8, 30])
  train(sess, loss, x_train, y_train, args=train_params, rng=rng)
  # Evaluate the accuracy of the MNIST model on legitimate test examples
  eval_params = {'batch_size': 128}
  accuracy = model_eval(sess, x, y, preds, x_test, y_test, args=eval_params)
  assert x_test.shape[0] == test_end - test_start, x_test.shape
  print('Test accuracy on legitimate test examples: {0}'.format(accuracy))
  report.clean_train_clean_eval = accuracy


IndentationError: ignored

In [ ]:
 source_samples=SOURCE_SAMPLES
  # Keep track of success (adversarial example classified in target)
 results = np.zeros((nb_classes, source_samples), dtype='i')
 perturbations = np.zeros((nb_classes, source_samples), dtype='f')
  # Initialize our array for grid visualization
 grid_shape = (nb_classes, nb_classes, img_rows, img_cols, nchannels)
 grid_viz_data = np.zeros(grid_shape, dtype='f')
    # Instantiate a SaliencyMapMethod attack object
 jsma = SaliencyMapMethod(model, sess=sess)
 jsma_params = {'theta': 1., 'gamma': 0.1,'clip_min': 0., 'clip_max': 1.,'y_target': None}
 figure = None
    # Loop over the samples we want to perturb into adversarial examples
 for sample_ind in xrange(0, source_samples):
    print('--------------------------------------')
    print('Attacking input %i/%i' % (sample_ind + 1, source_samples))
    sample = x_test[sample_ind:(sample_ind + 1)]
    current_class = int(np.argmax(y_test[sample_ind]))
    target_classes = other_classes(nb_classes, current_class)
    grid_viz_data[current_class, current_class, :, :, :] = np.reshape(sample, (img_rows, img_cols, nchannels))
    # Loop over all target classes
    for target in target_classes:
      print('Generating adv. example for target class %i' % target)
      # This call runs the Jacobian-based saliency map approach
      one_hot_target = np.zeros((1, nb_classes), dtype=np.float32)
      one_hot_target[0, target] = 1
      jsma_params['y_target'] = one_hot_target
      adv_x = jsma.generate_np(sample, **jsma_params)
      # Check if success was achieved
      res = int(model_argmax(sess, x, preds, adv_x) == target)
      # Compute number of modified features
      adv_x_reshape = adv_x.reshape(-1)
      test_in_reshape = x_test[sample_ind].reshape(-1)
      nb_changed = np.where(adv_x_reshape != test_in_reshape)[0].shape[0]
      percent_perturb = float(nb_changed) / adv_x.reshape(-1).shape[0]
      # Display the original and adversarial images side-by-side
      if viz_enabled:
        figure = pair_visual(np.reshape(sample, (img_rows, img_cols, nchannels)),np.reshape(adv_x, (img_rows, img_cols, nchannels)), figure)
      # Add our adversarial example to our grid data
      grid_viz_data[target, current_class, :, :, :] = np.reshape(adv_x, (img_rows, img_cols, nchannels))

      # Update the arrays for later analysis
      results[target, sample_ind] = res
      perturbations[target, sample_ind] = percent_perturb

 print('--------------------------------------')

 # Compute the number of adversarial examples that were successfully found
 nb_targets_tried = ((nb_classes - 1) * source_samples)
 succ_rate = float(np.sum(results)) / nb_targets_tried
 print('Avg. rate of successful adv. examples {0:.4f}'.format(succ_rate))
 report.clean_train_adv_eval = 1. - succ_rate

    # Compute the average distortion introduced by the algorithm
 percent_perturbed = np.mean(perturbations)
 print('Avg. rate of perturbed features {0:.4f}'.format(percent_perturbed))

    # Compute the average distortion introduced for successful samples only
 percent_perturb_succ = np.mean(perturbations * (results == 1))
 print('Avg. rate of perturbed features for successful '
          'adversarial examples {0:.4f}'.format(percent_perturb_succ))

    # Close TF session
 sess.close()

    # Finally, block & display a grid of all the adversarial examples
 if viz_enabled:
    import matplotlib.pyplot as plt
    plt.close(figure)
    _ = grid_visual(grid_viz_data)

return report


def main(argv=None):
    mnist_tutorial_jsma(viz_enabled=FLAGS.viz_enabled,
                        nb_epochs=FLAGS.nb_epochs,
                        batch_size=FLAGS.batch_size,
                        nb_classes=FLAGS.nb_classes,
                        source_samples=FLAGS.source_samples,
                        learning_rate=FLAGS.learning_rate)


if __name__ == '__main__':
    flags.DEFINE_boolean('viz_enabled', True, 'Visualize adversarial ex.')
    flags.DEFINE_integer('nb_epochs', 6, 'Number of epochs to train model')
    flags.DEFINE_integer('batch_size', 128, 'Size of training batches')
    flags.DEFINE_integer('nb_classes', 10, 'Number of output classes')
    flags.DEFINE_integer('source_samples', 10, 'Nb of test inputs to attack')
    flags.DEFINE_float('learning_rate', 0.001, 'Learning rate for training')

    tf.app.run()

/content/gdrive/My Drive/Colab Notebooks/cleverhans/cleverhans/attacks_tf.py:27: UserWarning: attacks_tf is deprecated and will be removed on 2019-07-18 or after. Code should import functions from their new locations directly.
  warnings.warn("attacks_tf is deprecated and will be removed on 2019-07-18"
[INFO 2021-06-03 20:18:48,830 cleverhans] Constructing new graph for attack SaliencyMapMethod


--------------------------------------
Attacking input 1/10
Generating adv. example for target class 0




/content/gdrive/My Drive/Colab Notebooks/cleverhans/cleverhans/compat.py:22: UserWarning: <function reduce_sum_v1 at 0x7f28c80f60e0> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "
/content/gdrive/My Drive/Colab Notebooks/cleverhans/cleverhans/compat.py:22: UserWarning: <function reduce_max_v1 at 0x7f28c80fe320> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "
/content/gdrive/My Drive/Colab Notebooks/cleverhans/cleverhans/compat.py:22: UserWarning: <f

FailedPreconditionError: ignored

In [ ]:
 source_samples=SOURCE_SAMPLES
  # Keep track of success (adversarial example classified in target)
 results = np.zeros((nb_classes, source_samples), dtype='i')
 perturbations = np.zeros((nb_classes, source_samples), dtype='f')
  # Initialize our array for grid visualization
 grid_shape = (nb_classes, nb_classes, img_rows, img_cols, nchannels)
 grid_viz_data = np.zeros(grid_shape, dtype='f')
    # Instantiate a SaliencyMapMethod attack object
 jsma = SaliencyMapMethod(model, sess=sess)
 jsma_params = {'theta': 1., 'gamma': 0.1,'clip_min': 0., 'clip_max': 1.,'y_target': None}
 figure = None
    # Loop over the samples we want to perturb into adversarial examples
 for sample_ind in xrange(0, source_samples):
    print('--------------------------------------')
    print('Attacking input %i/%i' % (sample_ind + 1, source_samples))
    sample = x_test[sample_ind:(sample_ind + 1)]
    current_class = int(np.argmax(y_test[sample_ind]))
    target_classes = other_classes(nb_classes, current_class)
    grid_viz_data[current_class, current_class, :, :, :] = np.reshape(sample, (img_rows, img_cols, nchannels))
    # Loop over all target classes
    for target in target_classes:
      print('Generating adv. example for target class %i' % target)
      # This call runs the Jacobian-based saliency map approach
      one_hot_target = np.zeros((1, nb_classes), dtype=np.float32)
      one_hot_target[0, target] = 1
      jsma_params['y_target'] = one_hot_target
      adv_x = jsma.generate_np(sample, **jsma_params)
      # Check if success was achieved
      res = int(model_argmax(sess, x, preds, adv_x) == target)
      # Compute number of modified features
      adv_x_reshape = adv_x.reshape(-1)
      test_in_reshape = x_test[sample_ind].reshape(-1)
      nb_changed = np.where(adv_x_reshape != test_in_reshape)[0].shape[0]
      percent_perturb = float(nb_changed) / adv_x.reshape(-1).shape[0]
      # Display the original and adversarial images side-by-side
      if viz_enabled:
        figure = pair_visual(np.reshape(sample, (img_rows, img_cols, nchannels)),np.reshape(adv_x, (img_rows, img_cols, nchannels)), figure)
      # Add our adversarial example to our grid data
      grid_viz_data[target, current_class, :, :, :] = np.reshape(adv_x, (img_rows, img_cols, nchannels))

      # Update the arrays for later analysis
      results[target, sample_ind] = res
      perturbations[target, sample_ind] = percent_perturb

 print('--------------------------------------')

  # Compute the number of adversarial examples that were successfully found
 nb_targets_tried = ((nb_classes - 1) * source_samples)
 succ_rate = float(np.sum(results)) / nb_targets_tried
 print('Avg. rate of successful adv. examples {0:.4f}'.format(succ_rate))
 report.clean_train_adv_eval = 1. - succ_rate
  # Compute the average distortion introduced by the algorithm
 percent_perturbed = np.mean(perturbations[np.where(perturbations!=0)])
 print('Avg. rate of perturbed features {0:.4f}'.format(percent_perturbed))
  # Compute the average distortion introduced for successful samples only
 percent_perturb_succ = np.mean(perturbations[np.where(perturbations!=0)] * (results[np.where(perturbations!=0)] == 1))
 print('Avg. rate of perturbed features for successful ','adversarial examples {0:.4f}'.format(percent_perturb_succ))
 sess.close()
  # Finally, block & display a grid of all the adversarial examples
 if viz_enabled:
    import matplotlib.pyplot as plt
    plt.close(figure)
    _ = grid_visual(grid_viz_data)
return report

def main(argv=None):
  from cleverhans_tutorials import check_installation
  check_installation(__file__)

  mnist_tutorial_jsma(viz_enabled=FLAGS.viz_enabled,
                      nb_epochs=FLAGS.nb_epochs,
                      batch_size=FLAGS.batch_size,
                      source_samples=FLAGS.source_samples,
                      learning_rate=FLAGS.learning_rate)


if __name__ == '__main__':
  flags.DEFINE_boolean('viz_enabled', VIZ_ENABLED,
                       'Visualize adversarial ex.')
  flags.DEFINE_integer('nb_epochs', NB_EPOCHS,
                       'Number of epochs to train model')
  flags.DEFINE_integer('batch_size', BATCH_SIZE, 'Size of training batches')
  flags.DEFINE_integer('source_samples', SOURCE_SAMPLES,
                       'Nb of test inputs to attack')
  flags.DEFINE_float('learning_rate', LEARNING_RATE,
                     'Learning rate for training')

  tf.app.run()

[INFO 2021-06-03 19:50:56,208 cleverhans] Constructing new graph for attack SaliencyMapMethod


--------------------------------------
Attacking input 1/10
Generating adv. example for target class 0


/content/gdrive/My Drive/Colab Notebooks/cleverhans/cleverhans/compat.py:22: UserWarning: <function reduce_sum_v1 at 0x7f776cb2c0e0> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "
/content/gdrive/My Drive/Colab Notebooks/cleverhans/cleverhans/compat.py:22: UserWarning: <function reduce_max_v1 at 0x7f776cb33320> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "
/content/gdrive/My Drive/Colab Notebooks/cleverhans/cleverhans/compat.py:22: UserWarning: <f

RuntimeError: ignored